#### Create ERA-5 MSE componets under /neelin2020/ERA-5_mse

In [1]:
import os
import sys
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime, date
from pathlib import Path
from numba import jit, vectorize, float64

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import warnings

In [2]:
warnings.filterwarnings('ignore')

In [ ]:
@jit(nopython=False)
def es_calc_bolton(temp):
    # in hPa

    tmelt  = 273.15
    tempc = temp - tmelt
    es = 6.112*np.exp(17.67*tempc/(243.5+tempc))

    return es

In [ ]:
@jit(nopython=False)
def es_calc(temp):
    """
    temp [x,p]
    """
    tmelt  = 273.15

    c0=0.6105851e+03
    c1=0.4440316e+02
    c2=0.1430341e+01
    c3=0.2641412e-01
    c4=0.2995057e-03
    c5=0.2031998e-05
    c6=0.6936113e-08
    c7=0.2564861e-11
    c8=-.3704404e-13

    tempc = temp - tmelt
    tempcorig = tempc

    #if tempc < -80:
    es_ltn80c = es_calc_bolton(temp)
    es_ltn80c = np.where(tempc < -80, es_ltn80c, 0)
    
    #else:
    es = c0+tempc*(c1+tempc*(c2+tempc*(c3+tempc*(c4+tempc*(c5+tempc*(c6+tempc*(c7+tempc*c8)))))))
    es_gtn80c = es/100
    es_gtn80c = np.where(tempc >= -80, es_gtn80c, 0)

    # complete es
    es = es_ltn80c + es_gtn80c

    return es

In [ ]:
@jit(nopython=False)
def qs_calc(temp, p_level):

    tmelt  = 273.15
    RV=461.5
    RD=287.04

    EPS=RD/RV

    press = p_level * 100. # in Pa
    tempc = temp - tmelt

    es=es_calc(temp) # [x,y,p]
    es=es * 100. #hPa
    
    qs = (EPS * es) / (press + ((EPS-1.)*es))

    return qs

In [ ]:
year_list = np.arange(2001,2010)

for year in year_list[:1]:

    era5_dir = Path('/neelin2020/ERA-5/NC_FILES/{}'.format(year))
    out_dir = Path('/neelin2020/ERA-5_mse/{}'.format(year))
    if out_dir.exists() == False:
        os.system('mkdir -p {}'.format(out_dir))
        print('Create new directory: {}'.format(out_dir))

    for mon in np.arange(1,13):

        ds_z = xr.open_dataset(era5_dir / 'era-5.z.{}.{}.nc'.format(year,str(mon).zfill(2))).rename({'valid_time':'time'})
        ds_t = xr.open_dataset(era5_dir / 'era-5.T.{}.{}.nc'.format(year,str(mon).zfill(2)))
        ds_q = xr.open_dataset(era5_dir / 'era-5.q.{}.{}.nc'.format(year,str(mon).zfill(2)))

        # calculate MSE at 850hPa and MSE_sat 500hPa 
        cp = 1004.
        Lv = 2.5e6

        # get qsat at 500hPa
        p_level = 500 
        T_500 = ds_t.t.sel(level=500).values
        qsat_500 = qs_calc(T_500, p_level)    

        # MSE at 850hPa 
        mse_850 = ( cp*ds_t.t.sel(level=850) + Lv*ds_q.q.sel(level=850) + ds_z.z.sel(pressure_level=850) )/1000 # KJ/kg
        # MSE_sat at 500hPa
        mse_sat_500 = ( cp*ds_t.t.sel(level=500) + Lv*qsat_500 + ds_z.z.sel(pressure_level=500) )/1000 # KJ/kg
        mse_850 = mse_850.rename('mse_850')
        mse_sat_500 = mse_sat_500.rename('mse_sat_500')

        # save into the directory
        ds_merged = xr.merge([mse_850, mse_sat_500],compat='override').drop(['level','pressure_level','expver','number']) # only avialable regions
        ds_merged.attrs['unit'] = 'KJ/kg' 
        ds_merged.attrs['created date'] = str(date.today())

        ds_merged.to_netcdf(out_dir / 'ERA-5.mse.{}.{}.nc'.format(year,str(mon).zfill(2)) )
        print(out_dir / 'ERA-5.mse.{}.{}.nc'.format(year,str(mon).zfill(2)))

In [3]:
test = xr.open_dataset('/neelin2020/ERA-5/NC_FILES/2002/era-5.z.2002.01.nc')

In [6]:
dim_vars = [i for i in test.dims]

In [9]:
if 'test' in dim_vars:
    print('yes')

In [ ]:
test = xr.open_dataset(out_dir / 'ERA-5.mse.{}.{}.nc'.format(year,str(mon).zfill(2)))

In [ ]:
test.mse_850.isel(time=0).plot()